In [1]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_2689622d3bb44c6295e7bc1d0783dde4(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_27f432c403a506fc9b89694a383d87f4282a96de','domain': {'id': 'ce9588cdb66f4bc8889c950585780ce9'},
            'password': 'l^R6K]-EMg&56c[/'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_train = pd.read_csv(get_object_storage_file_with_credentials_2689622d3bb44c6295e7bc1d0783dde4('DefaultProjectnchellaminibmcom', 'train_moviesentiment.csv'))
df_train.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [2]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
PhraseId      156060 non-null int64
SentenceId    156060 non-null int64
Phrase        156060 non-null object
Sentiment     156060 non-null int64
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [3]:
sentiment=df_train['Sentiment']

In [4]:
sentiment.head()

0    1
1    2
2    2
3    2
4    2
Name: Sentiment, dtype: int64

In [5]:
import nltk

In [6]:
from nltk import word_tokenize

In [7]:
import string
table=str.maketrans('','',string.punctuation)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
vect=TfidfVectorizer()

In [10]:
train=vect.fit_transform(df_train['Phrase'])


In [11]:
vect.vocabulary_

{'mad': 8121,
 'reminiscence': 10989,
 'circle': 2354,
 'principals': 10296,
 'scooter': 11645,
 'sophomoric': 12457,
 'review': 11173,
 'travis': 13858,
 'kirshner': 7525,
 'spoken': 12608,
 'exhibits': 4736,
 'exploiting': 4791,
 'brief': 1734,
 'embarrassment': 4351,
 'overstuffed': 9461,
 'bloodwork': 1500,
 'cagney': 1919,
 'why': 14926,
 'cooper': 2924,
 'hi': 6331,
 'reports': 11032,
 'mystique': 8880,
 'passe': 9628,
 'unstoppable': 14370,
 'comatose': 2571,
 'firmer': 5151,
 'captured': 2004,
 'debated': 3338,
 'able': 160,
 'obscenity': 9183,
 'regain': 10895,
 'avengers': 1015,
 'spiteful': 12589,
 'whoopee': 14922,
 'whirl': 14904,
 'mart': 8273,
 'immune': 6715,
 'errors': 4573,
 'plympton': 10022,
 'woven': 15105,
 'senegalese': 11788,
 'reception': 10782,
 'negated': 8979,
 'dread': 4047,
 'honorable': 6455,
 'chuckle': 2329,
 'mexico': 8493,
 'her': 6306,
 'projector': 10373,
 'senses': 11795,
 'brecht': 1718,
 'row': 11382,
 'yourselves': 15192,
 'enjoyable': 4470,
 's

In [12]:
train.shape

(156060, 15240)

In [13]:
vect.idf_

array([  9.77994841,   7.76504538,   8.69532236, ...,  10.18541351,
        10.65541714,  11.16624277])

In [14]:
train_arr=train.toarray()

In [15]:
train_arr.shape

(156060, 15240)

In [16]:
from sklearn.model_selection import train_test_split

In [45]:
x_train,x_test,y_train,y_test=train_test_split(df_train['Phrase'],sentiment,test_size=0.3,random_state=1)

In [18]:
x_train.shape

(117045,)

In [47]:
x_train=vect.fit_transform(x_train)

In [48]:
x_train.shape

(109242, 15215)

# # It is very important to note that x_test should only be transformed but not not fit_transform... because the x_test should only be transformed based on x_train vocabulary.

## If x_test is also fit_tranformed then the shape of the vocabulary in train and test will be different

In [49]:
x_test=vect.transform(x_test)

In [50]:
x_test.shape

(46818, 15215)

In [23]:
from sklearn import svm

In [24]:
cls=svm.SVC()

In [25]:
cls.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [26]:
x_train.shape

(117045, 15223)

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
y_pred=cls.predict(x_test)

In [29]:
x_test.shape

(39015, 15223)

In [30]:
x_train.shape

(117045, 15223)

In [36]:
y_pred[1821]

2

In [37]:
accuracy_score(y_test, y_pred, normalize=True)

0.50916314238113547

In [38]:
import xgboost as xgb

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [39]:

from xgboost import XGBClassifier

In [40]:
model=XGBClassifier()

In [51]:
model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [52]:
y_pred1=model.predict(x_test)

In [53]:
y_pred1.shape

(46818,)

In [54]:
accuracy_score(y_test, y_pred1)

0.53408945277457387

In [57]:
y_pred1[12001]

2

In [58]:
from sklearn.naive_bayes import MultinomialNB

In [59]:
model_naive=MultinomialNB()

In [60]:
model_naive.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [61]:
y_pred2=model_naive.predict(x_test)

In [62]:
y_pred2

array([2, 1, 2, ..., 2, 3, 2])

In [63]:
accuracy_score(y_test,y_pred2)

0.58537314708018284

In [65]:
from sklearn.ensemble import RandomForestClassifier

In [66]:
model_random=RandomForestClassifier(max_depth=3, random_state=0)

In [67]:
model_random.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [68]:
y_pred3=model_random.predict(x_test)

In [69]:
y_pred3

array([2, 2, 2, ..., 2, 2, 2])

In [70]:
accuracy_score(y_test,y_pred3)

0.50869323764364138